reference: https://www.kaggle.com/code/toshikazuwatanabe/connect4-make-submission-with-stable-baselines3

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import torch as th
from torch import nn as nn
import torch.nn.functional as F

from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import load_results
from stable_baselines3.common.torch_layers import NatureCNN
from stable_baselines3.common.policies import ActorCriticPolicy, ActorCriticCnnPolicy
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

In [ ]:
LOG_DIR = os.path.join(os.getcwd(), 'logs')	# トレーニングのログを保存するディレクトリ
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
# 環境の作成
from environment import ConnectFourGym

env = ConnectFourGym()
env

In [ ]:
# ログを取得する
env = Monitor(env, LOG_DIR, allow_early_resets=True)
env

In [ ]:
# 「DummyVecEnv」は、OpenAI Gymの環境をベクトル化するための特殊なラッパーです。
# 通常、強化学習アルゴリズムは一度に1つの環境しか処理できませんが、これを使用することで
# 複数の環境を同時に実行することができます。これにより、学習プロセスが効率的になります。
env = DummyVecEnv([lambda: env])
env